In [4]:
import math, random, fractions
from qiskit import *
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit_ibm_provider import IBMProvider

In [5]:
def create_circuit(num_bits):
    circuit = QuantumCircuit(num_bits, num_bits)
    circuit.h(range(num_bits)) 
    return circuit


In [6]:
def fourier_inverse(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    
    for j in range(n):
        for m in range(j):
            angle = -math.pi/float(2**(j-m))
            circuit.cp(angle, m, j)
        circuit.h(j)
    circuit.barrier()
    

In [7]:
def modular_exponentiation(circuit, n, a, N):
    for x in range(n):
        exponent = 2 ** x 
        a_exp = (a**exponent) % N 

        for y in range(n):
            if y != x and (a_exp >> y) & 1:
                angle = math.pi / exponent
                circuit.cp(angle, x, y)
    circuit.barrier()


In [8]:
def compute_private_key(p, q, e):
    phi_n = (p - 1) * (q - 1) 
    d = pow(e, -1, phi_n)
    return d

In [9]:
def shors_algorithm(N):
    n = math.ceil(math.log(N, 2))
    circuit = create_circuit(n)

    a = random.randint(2, N - 1)
    while math.gcd(a, N) != 1:
        a = random.randint(2, N - 1)

    modular_exponentiation(circuit, n, a, N)
    fourier_inverse(circuit, n) 
    circuit.measure(range(n), range(n))
    
    
    simulator = Aer.get_backend('aer_simulator')
    result = execute(circuit,simulator, shots=1024).result()
    
#     provider = IBMProvider()
#     provider.active_account()
#     qcomp = provider.get_backend('ibm_kyoto')
#     job = execute(circuit, backend = qcomp)
#     job_monitor(job)
#     result = job.result()
    
    counts = result.get_counts(circuit)
    

    most_probable = int(max(counts, key=counts.get), 2)
    phase = most_probable / (2**n)
    frac = fractions.Fraction(phase).limit_denominator(N)
    r = frac.denominator
    
    p = math.gcd(pow(a, r//2) - 1, N)
    q = math.gcd(pow(a, r//2) + 1, N)
    return p, q, circuit


In [10]:
def main():
    N = 15
    e = 65537
    found = False

    while not found:
            result = shors_algorithm(N)
            if result:
                p, q, circuit = result
                if p != 1 and q != 1 and p * q == N:
                    print(f"Factors of {N}: {p} and {q}")
                    d = compute_private_key(p, q, e)
                    print(f"Private key: {d}")
                    display(circuit.draw())
                    found = True

In [11]:
if __name__ == "__main__":
    main()


Factors of 15: 5 and 3
Private key: 1


┌───┐                                          ░    ┌───┐               »
q_0: ┤ H ├─■──────■──────■────────■────────■────────░──X─┤ H ├─■─────────────»
     ├───┤ │P(π)  │      │P(π/2)  │        │        ░  │ └───┘ │P(-π/2) ┌───┐»
q_1: ┤ H ├─■──────┼──────■────────┼────────┼────────░──┼───X───■────────┤ H ├»
     ├───┤        │               │P(π/4)  │        ░  │   │            └───┘»
q_2: ┤ H ├────────┼───────────────■────────┼────────░──┼───X─────────────────»
     ├───┤        │P(π)                    │P(π/8)  ░  │                     »
q_3: ┤ H ├────────■────────────────────────■────────░──X─────────────────────»
     └───┘                                          ░                        »
c: 4/════════════════════════════════════════════════════════════════════════»
                                                                             »
«                                                                  ░ ┌─┐      »
«q_0: ─■───────────────────■───────────────────────────────────────░─┤M├──────»
«      │                   │                                       ░ └╥┘┌─┐   »
«q_1: ─┼─────────■─────────┼──────────────■────────────────────────░──╫─┤M├───»
«      │P(-π/4)  │P(-π/2)  │        ┌───┐ │                        ░  ║ └╥┘┌─┐»
«q_2: ─■─────────■─────────┼────────┤ H ├─┼─────────■──────────────░──╫──╫─┤M├»
«                          │P(-π/8) └───┘ │P(-π/4)  │P(-π/2) ┌───┐ ░  ║  ║ └╥┘»
«q_3: ─────────────────────■──────────────■─────────■────────┤ H ├─░──╫──╫──╫─»
«                                                            └───┘ ░  ║  ║  ║ »
«c: 4/════════════════════════════════════════════════════════════════╩══╩══╩═»
«                                                                     0  1  2 »
«        
«q_0: ───
«        
«q_1: ───
«        
«q_2: ───
«     ┌─┐
«q_3: ┤M├
«     └╥┘
«c: 4/═╩═
«      3